In [3]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('basics').getOrCreate()

In [4]:
df=spark.read.json('people.json')

In [5]:
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [7]:
df.printSchema()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



In [8]:
df.columns

['age', 'name']

In [12]:
df.describe().show()

+-------+------------------+-------+
|summary|               age|   name|
+-------+------------------+-------+
|  count|                 2|      3|
|   mean|              24.5|   null|
| stddev|7.7781745930520225|   null|
|    min|                19|   Andy|
|    max|                30|Michael|
+-------+------------------+-------+



In [18]:
#setting age column to int
from pyspark.sql.types import StructType,StructField,StringType,IntegerType

In [19]:
#param name,col name and nullable
data_schema=[StructField("age",IntegerType(),True),StructField(("name"),StringType(),True)]
final_struct=StructType(fields=data_schema)

In [20]:
df=spark.read.json('people.json',schema=final_struct)
df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- name: string (nullable = true)



### Grabbing data

In [22]:
df['age']

Column<b'age'>

In [23]:
type(df['age'])

pyspark.sql.column.Column

In [25]:
df.select('age').show()

+----+
| age|
+----+
|null|
|  30|
|  19|
+----+



In [26]:
type(df.select('age'))

pyspark.sql.dataframe.DataFrame

In [28]:
df.select(df['age']).show()

+----+
| age|
+----+
|null|
|  30|
|  19|
+----+



In [29]:
df.head(2)

[Row(age=None, name='Michael'), Row(age=30, name='Andy')]

In [32]:
df.select(['age','name']).show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [33]:
df.select([df['age'],df['name']]).show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



### Creating new columns

In [35]:
#adding a new col
df.withColumn('doublenewAge',df['age']*2).show()

+----+-------+------------+
| age|   name|doublenewAge|
+----+-------+------------+
|null|Michael|        null|
|  30|   Andy|          60|
|  19| Justin|          38|
+----+-------+------------+



In [36]:
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [39]:
df.withColumnRenamed('age','new_age').show()

+-------+-------+
|new_age|   name|
+-------+-------+
|   null|Michael|
|     30|   Andy|
|     19| Justin|
+-------+-------+



In [40]:
df.withColumn('age+1',df['age']+1).show()

+----+-------+-----+
| age|   name|age+1|
+----+-------+-----+
|null|Michael| null|
|  30|   Andy|   31|
|  19| Justin|   20|
+----+-------+-----+



### Using SQL

In [41]:
#to use sql queries we need to register it to temporary view
df.createOrReplaceTempView("people")

In [42]:
spark.sql("select * from people").show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [43]:
spark.sql('select * from people where age = 30').show()

+---+----+
|age|name|
+---+----+
| 30|Andy|
+---+----+



###  Basic operations

In [45]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("operatiosn").getOrCreate()

In [46]:
df=spark.read.csv("appl_stock.csv",inferSchema=True,header=True)

In [47]:
df.printSchema()

root
 |-- Date: timestamp (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



In [51]:
#filtering data
df.head()

Row(Date=datetime.datetime(2010, 1, 4, 0, 0), Open=213.429998, High=214.499996, Low=212.38000099999996, Close=214.009998, Volume=123432400, Adj Close=27.727039)

In [52]:
df.filter("Close<500").show()

+-------------------+------------------+------------------+------------------+------------------+---------+------------------+
|               Date|              Open|              High|               Low|             Close|   Volume|         Adj Close|
+-------------------+------------------+------------------+------------------+------------------+---------+------------------+
|2010-01-04 00:00:00|        213.429998|        214.499996|212.38000099999996|        214.009998|123432400|         27.727039|
|2010-01-05 00:00:00|        214.599998|        215.589994|        213.249994|        214.379993|150476200|27.774976000000002|
|2010-01-06 00:00:00|        214.379993|            215.23|        210.750004|        210.969995|138040000|27.333178000000004|
|2010-01-07 00:00:00|            211.75|        212.000006|        209.050005|            210.58|119282800|          27.28265|
|2010-01-08 00:00:00|        210.299994|        212.000006|209.06000500000002|211.98000499999998|111902700|    

In [53]:
df.filter("Close<500").select(['Close','Open']).show()

+------------------+------------------+
|             Close|              Open|
+------------------+------------------+
|        214.009998|        213.429998|
|        214.379993|        214.599998|
|        210.969995|        214.379993|
|            210.58|            211.75|
|211.98000499999998|        210.299994|
|210.11000299999998|212.79999700000002|
|        207.720001|209.18999499999998|
|        210.650002|        207.870005|
|            209.43|210.11000299999998|
|            205.93|210.92999500000002|
|        215.039995|        208.330002|
|            211.73|        214.910006|
|        208.069996|        212.079994|
|            197.75|206.78000600000001|
|        203.070002|202.51000200000001|
|        205.940001|205.95000100000001|
|        207.880005|        206.849995|
|        199.289995|        204.930004|
|        192.060003|        201.079996|
|        194.729998|192.36999699999998|
+------------------+------------------+
only showing top 20 rows



In [58]:
#another method
df.filter(df['Close']<500).select(["close","open"]).show()

+------------------+------------------+
|             close|              open|
+------------------+------------------+
|        214.009998|        213.429998|
|        214.379993|        214.599998|
|        210.969995|        214.379993|
|            210.58|            211.75|
|211.98000499999998|        210.299994|
|210.11000299999998|212.79999700000002|
|        207.720001|209.18999499999998|
|        210.650002|        207.870005|
|            209.43|210.11000299999998|
|            205.93|210.92999500000002|
|        215.039995|        208.330002|
|            211.73|        214.910006|
|        208.069996|        212.079994|
|            197.75|206.78000600000001|
|        203.070002|202.51000200000001|
|        205.940001|205.95000100000001|
|        207.880005|        206.849995|
|        199.289995|        204.930004|
|        192.060003|        201.079996|
|        194.729998|192.36999699999998|
+------------------+------------------+
only showing top 20 rows



In [61]:
df.filter((df['Close']<500) & (df['open']>200)).show()

+-------------------+------------------+------------------+------------------+------------------+---------+------------------+
|               Date|              Open|              High|               Low|             Close|   Volume|         Adj Close|
+-------------------+------------------+------------------+------------------+------------------+---------+------------------+
|2010-01-04 00:00:00|        213.429998|        214.499996|212.38000099999996|        214.009998|123432400|         27.727039|
|2010-01-05 00:00:00|        214.599998|        215.589994|        213.249994|        214.379993|150476200|27.774976000000002|
|2010-01-06 00:00:00|        214.379993|            215.23|        210.750004|        210.969995|138040000|27.333178000000004|
|2010-01-07 00:00:00|            211.75|        212.000006|        209.050005|            210.58|119282800|          27.28265|
|2010-01-08 00:00:00|        210.299994|        212.000006|209.06000500000002|211.98000499999998|111902700|    

In [63]:
    df.filter(df['low']==197.16).show()

+-------------------+------------------+----------+------+------+---------+---------+
|               Date|              Open|      High|   Low| Close|   Volume|Adj Close|
+-------------------+------------------+----------+------+------+---------+---------+
|2010-01-22 00:00:00|206.78000600000001|207.499996|197.16|197.75|220441900|25.620401|
+-------------------+------------------+----------+------+------+---------+---------+



In [64]:
df.filter(df['low']==197.16).collect()

[Row(Date=datetime.datetime(2010, 1, 22, 0, 0), Open=206.78000600000001, High=207.499996, Low=197.16, Close=197.75, Volume=220441900, Adj Close=25.620401)]

In [66]:
type(df.filter(df['low']==197.16).collect()[0])

pyspark.sql.types.Row

In [69]:
#rows can be converted to dict
result=df.filter(df['low']==197.16).collect()[0]
result.asDict()

{'Date': datetime.datetime(2010, 1, 22, 0, 0),
 'Open': 206.78000600000001,
 'High': 207.499996,
 'Low': 197.16,
 'Close': 197.75,
 'Volume': 220441900,
 'Adj Close': 25.620401}

In [70]:
result.High

207.499996

In [71]:
for item in result:
    print(item)

2010-01-22 00:00:00
206.78000600000001
207.499996
197.16
197.75
220441900
25.620401


## Groupby and aggregate functions

In [72]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("groupbyAgg").getOrCreate()

In [73]:
df=spark.read.csv('sales_info.csv',inferSchema=True,header=True)

In [74]:
df.printSchema()

root
 |-- Company: string (nullable = true)
 |-- Person: string (nullable = true)
 |-- Sales: double (nullable = true)



In [75]:
df.show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|    Sam|200.0|
|   GOOG|Charlie|120.0|
|   GOOG|  Frank|340.0|
|   MSFT|   Tina|600.0|
|   MSFT|    Amy|124.0|
|   MSFT|Vanessa|243.0|
|     FB|   Carl|870.0|
|     FB|  Sarah|350.0|
|   APPL|   John|250.0|
|   APPL|  Linda|130.0|
|   APPL|   Mike|750.0|
|   APPL|  Chris|350.0|
+-------+-------+-----+



In [81]:
df.groupBy("Company")

In [84]:
df.groupBy("company").mean().show()

+-------+-----------------+
|company|       avg(Sales)|
+-------+-----------------+
|   APPL|            370.0|
|   GOOG|            220.0|
|     FB|            610.0|
|   MSFT|322.3333333333333|
+-------+-----------------+



In [85]:
df.groupBy("company").count().show()

+-------+-----+
|company|count|
+-------+-----+
|   APPL|    4|
|   GOOG|    3|
|     FB|    2|
|   MSFT|    3|
+-------+-----+



In [86]:
df.groupBy("company").max().show()

+-------+----------+
|company|max(Sales)|
+-------+----------+
|   APPL|     750.0|
|   GOOG|     340.0|
|     FB|     870.0|
|   MSFT|     600.0|
+-------+----------+



In [90]:
#using aggregate methods
df.agg({'Sales':'max'}).show()

+----------+
|max(Sales)|
+----------+
|     870.0|
+----------+



In [94]:
df.groupBy("company").agg({'sales':'min'}).show()

+-------+----------+
|company|min(sales)|
+-------+----------+
|   APPL|     130.0|
|   GOOG|     120.0|
|     FB|     350.0|
|   MSFT|     124.0|
+-------+----------+



### functions

In [102]:
from pyspark.sql.functions import countDistinct,avg,stddev,format_number

In [103]:
df.show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|    Sam|200.0|
|   GOOG|Charlie|120.0|
|   GOOG|  Frank|340.0|
|   MSFT|   Tina|600.0|
|   MSFT|    Amy|124.0|
|   MSFT|Vanessa|243.0|
|     FB|   Carl|870.0|
|     FB|  Sarah|350.0|
|   APPL|   John|250.0|
|   APPL|  Linda|130.0|
|   APPL|   Mike|750.0|
|   APPL|  Chris|350.0|
+-------+-------+-----+



In [104]:
df.select(countDistinct('Sales')).show()

+---------------------+
|count(DISTINCT Sales)|
+---------------------+
|                   11|
+---------------------+



In [105]:
#change name using alias
df.select(countDistinct('Sales').alias('sales_new')).show()

+---------+
|sales_new|
+---------+
|       11|
+---------+



In [106]:
df.select(avg('sales')).show()

+-----------------+
|       avg(sales)|
+-----------------+
|360.5833333333333|
+-----------------+



In [107]:
df.select(stddev('sales')).show()

+------------------+
|stddev_samp(sales)|
+------------------+
|250.08742410799007|
+------------------+



In [111]:
df.select(format_number(stddev('sales'),2).alias('std')).show()

+------+
|   std|
+------+
|250.09|
+------+



In [112]:
#using order by
df.show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|    Sam|200.0|
|   GOOG|Charlie|120.0|
|   GOOG|  Frank|340.0|
|   MSFT|   Tina|600.0|
|   MSFT|    Amy|124.0|
|   MSFT|Vanessa|243.0|
|     FB|   Carl|870.0|
|     FB|  Sarah|350.0|
|   APPL|   John|250.0|
|   APPL|  Linda|130.0|
|   APPL|   Mike|750.0|
|   APPL|  Chris|350.0|
+-------+-------+-----+



In [113]:
df.orderBy('Company').show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   APPL|  Chris|350.0|
|   APPL|   John|250.0|
|   APPL|  Linda|130.0|
|   APPL|   Mike|750.0|
|     FB|   Carl|870.0|
|     FB|  Sarah|350.0|
|   GOOG|  Frank|340.0|
|   GOOG|    Sam|200.0|
|   GOOG|Charlie|120.0|
|   MSFT|   Tina|600.0|
|   MSFT|    Amy|124.0|
|   MSFT|Vanessa|243.0|
+-------+-------+-----+



In [114]:
df.orderBy(df['Company']).show()


+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   APPL|  Chris|350.0|
|   APPL|   John|250.0|
|   APPL|  Linda|130.0|
|   APPL|   Mike|750.0|
|     FB|   Carl|870.0|
|     FB|  Sarah|350.0|
|   GOOG|  Frank|340.0|
|   GOOG|    Sam|200.0|
|   GOOG|Charlie|120.0|
|   MSFT|   Tina|600.0|
|   MSFT|    Amy|124.0|
|   MSFT|Vanessa|243.0|
+-------+-------+-----+



In [115]:
df.orderBy(df['Company'].desc()).show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   MSFT|    Amy|124.0|
|   MSFT|   Tina|600.0|
|   MSFT|Vanessa|243.0|
|   GOOG|    Sam|200.0|
|   GOOG|  Frank|340.0|
|   GOOG|Charlie|120.0|
|     FB|  Sarah|350.0|
|     FB|   Carl|870.0|
|   APPL|   John|250.0|
|   APPL|  Linda|130.0|
|   APPL|  Chris|350.0|
|   APPL|   Mike|750.0|
+-------+-------+-----+



### dates and timestamp

In [116]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("dates").getOrCreate()

In [117]:
df=spark.read.csv("appl_stock.csv",header=True,inferSchema=True)

In [119]:
df.show(5)

+-------------------+----------+----------+------------------+------------------+---------+------------------+
|               Date|      Open|      High|               Low|             Close|   Volume|         Adj Close|
+-------------------+----------+----------+------------------+------------------+---------+------------------+
|2010-01-04 00:00:00|213.429998|214.499996|212.38000099999996|        214.009998|123432400|         27.727039|
|2010-01-05 00:00:00|214.599998|215.589994|        213.249994|        214.379993|150476200|27.774976000000002|
|2010-01-06 00:00:00|214.379993|    215.23|        210.750004|        210.969995|138040000|27.333178000000004|
|2010-01-07 00:00:00|    211.75|212.000006|        209.050005|            210.58|119282800|          27.28265|
|2010-01-08 00:00:00|210.299994|212.000006|209.06000500000002|211.98000499999998|111902700|         27.464034|
+-------------------+----------+----------+------------------+------------------+---------+------------------+
o

In [120]:
from pyspark.sql.functions import format_number,dayofmonth,dayofweek,dayofyear,hour,month,year,weekofyear,date_format

In [124]:
df.select(hour(df['Date'])).show(5)

+----------+
|hour(Date)|
+----------+
|         0|
|         0|
|         0|
|         0|
|         0|
+----------+
only showing top 5 rows



In [125]:
df.select(dayofmonth(df['Date'])).show(5)

+----------------+
|dayofmonth(Date)|
+----------------+
|               4|
|               5|
|               6|
|               7|
|               8|
+----------------+
only showing top 5 rows



In [126]:
df.select(month('Date')).show(5)

+-----------+
|month(Date)|
+-----------+
|          1|
|          1|
|          1|
|          1|
|          1|
+-----------+
only showing top 5 rows



### So for example, let's say we wanted to know the average closing price per year. Easy! With a groupby and the year() function call:

In [128]:
df.show(3)

+-------------------+----------+----------+------------------+----------+---------+------------------+
|               Date|      Open|      High|               Low|     Close|   Volume|         Adj Close|
+-------------------+----------+----------+------------------+----------+---------+------------------+
|2010-01-04 00:00:00|213.429998|214.499996|212.38000099999996|214.009998|123432400|         27.727039|
|2010-01-05 00:00:00|214.599998|215.589994|        213.249994|214.379993|150476200|27.774976000000002|
|2010-01-06 00:00:00|214.379993|    215.23|        210.750004|210.969995|138040000|27.333178000000004|
+-------------------+----------+----------+------------------+----------+---------+------------------+
only showing top 3 rows



In [129]:
df.select(year('Date')).show(3)

+----------+
|year(Date)|
+----------+
|      2010|
|      2010|
|      2010|
+----------+
only showing top 3 rows



In [133]:
new_df=df.withColumn('year',year('Date'))
new_df.show(4)

+-------------------+----------+----------+------------------+----------+---------+------------------+----+
|               Date|      Open|      High|               Low|     Close|   Volume|         Adj Close|year|
+-------------------+----------+----------+------------------+----------+---------+------------------+----+
|2010-01-04 00:00:00|213.429998|214.499996|212.38000099999996|214.009998|123432400|         27.727039|2010|
|2010-01-05 00:00:00|214.599998|215.589994|        213.249994|214.379993|150476200|27.774976000000002|2010|
|2010-01-06 00:00:00|214.379993|    215.23|        210.750004|210.969995|138040000|27.333178000000004|2010|
|2010-01-07 00:00:00|    211.75|212.000006|        209.050005|    210.58|119282800|          27.28265|2010|
+-------------------+----------+----------+------------------+----------+---------+------------------+----+
only showing top 4 rows



In [154]:
ans_df=new_df.groupBy('year').agg({"Close":'avg'})
ans_df.select('year',format_number('avg(Close)',2).alias('mean_close')).show()

+----+----------+
|year|mean_close|
+----+----------+
|2015|    120.04|
|2013|    472.63|
|2014|    295.40|
|2012|    576.05|
|2016|    104.60|
|2010|    259.84|
|2011|    364.00|
+----+----------+



In [162]:
#another method
new_df.groupBy('year').mean().select('year','avg(Close)').show()

+----+------------------+
|year|        avg(Close)|
+----+------------------+
|2015|120.03999980555547|
|2013| 472.6348802857143|
|2014| 295.4023416507935|
|2012| 576.0497195640002|
|2016|104.60400786904763|
|2010| 259.8424600000002|
|2011|364.00432532142867|
+----+------------------+



### Missing values

In [163]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('missingdata').getOrCreate()

In [165]:
df=spark.read.csv('ContainsNull.csv',header=True,inferSchema=True)
df.show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp2| null| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [166]:
df.na.drop().show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp4|Cindy|456.0|
+----+-----+-----+



In [167]:
#has to have 2 non null values
df.na.drop(thresh=2).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [168]:
df.na.drop(subset=('Sales')).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [169]:
df.na.drop(how='any').show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp4|Cindy|456.0|
+----+-----+-----+



In [170]:
df.na.drop(how='all').show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp2| null| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [171]:
#filling the missing values
df.show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp2| null| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [172]:
df.na.fill("new_val").show()

+----+-------+-----+
|  Id|   Name|Sales|
+----+-------+-----+
|emp1|   John| null|
|emp2|new_val| null|
|emp3|new_val|345.0|
|emp4|  Cindy|456.0|
+----+-------+-----+



In [173]:
df.na.fill(0).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|  0.0|
|emp2| null|  0.0|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [201]:
#fill in specific cols
df.na.fill(0,subset=['Sales']).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|  0.0|
|emp2| null|  0.0|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [206]:
from pyspark.sql.functions import mean
mean_val=df.select(mean(df['Sales'])).collect()

In [217]:
mean_val

[Row(avg(Sales)=400.5)]

In [218]:
ans=mean_val[0][0]
df.na.fill(ans,["Sales"]).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|400.5|
|emp2| null|400.5|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [219]:
#one liner
df.na.fill(df.select(mean(df['Sales'])).collect()[0][0],['Sales']).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|400.5|
|emp2| null|400.5|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+

